<h1><b>Statistique en Bioinformatique : </b> TME solo </h1><br>
<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><b>Soumission</b></p>
<ul>
<li>Renommer le fichier TMEsolo_2023.ipynb pour NomEtudiant_TMEsolo.ipynb </li>
<li>Envoyer via moodle</li>
</ul>
</div>
<div class="alert alert-warning" role="alert" style="margin: 10px">

</div>

<h2> Recherche de Motifs</h2>
Nous allons développer un algorithme très simple pour la recherche de motifs dans des séquences d'ADN et comparer les résultats avec l'algorithme Expectation et Maximization. 

In [124]:
# Configuration

import numpy as np
import matplotlib.pyplot as plt
import random

nuc = ['A', 'C', 'G', 'T']



<b>1. Génération des séquences aléatoires </b>:
Nous allons d'abord générer des données artificielles, un jeu de données de séquences ayant le motif (positives). Pour générer les séquences positives, 
utilisez la matrice de fréquence ci-dessous, la séquence consensus de ce motif "AGTATAG".

In [125]:


k = 7 #taille du motif
N = 100 #nombre de sequences

MotifConsensus = "AGTATAG"

freq = np.array([
  [1000,  12,   3,  1000,   5,   1000,   1],
  [10,    8,   3,   3,   0,    10,   1],
  [4,   1000,   2,   7,   5,    13,   1000],
  [6,    10,  1000,   5,   1000,   6,   1]])


<b> Générer de séquences positives </b>:
Écrivez une fonction qui génère N séquences de taille T contenant le motif "AGTATAG". 
La séquence du motif doit être généré en utilisant la matrice de Probabilité produite à partir de la matrice de fréquence.
La position d'insertion du motif est aléatoire et ses residues sont en **majuscules**, 
les autres residues de la séquence hors le motif sont représentés en minuscule. 



In [126]:

def createPosSequences(freq, nuc, k=7, N=100, T=100):
    """
    Genere N sequences positives (ayant le motif)
    entrée freq : matrice de frequences qui represent le motif d'interet
    entrée nuc  : alphabet 
    entrée k    : taille du motif
    entrée N    : nombre de sequences
    entrée T    : taille de chaque sequence
    sortie posMotif  : list contenant les sequences positives, motif implanté
    sortie motifs    : list contenant les motifs selement
    """

    motif = ""
    motifs =  []
    posMotif = []

    #Récupération du motif
    index_max_motif = [np.argmax(freq[:,i]) for i in range(k)]
    for i in range(k):
        motif+=nuc[index_max_motif[i]]
    
    for n in range(N):
        
        #Index d'ajout du motif
        index_demarrage= np.random.randint(T-k)
        seq_avant = ""
        seq_apres = ""
        seq = ""
        
        #Ajout de la premiere partie
        for i in range(index_demarrage):
            seq_avant+=nuc[np.random.randint(len(nuc))]        
                
        #Ajout du reste de la séquence
        for i in range(T-index_demarrage-k):
            seq_apres+=nuc[np.random.randint(len(nuc))]
        
        #Création de la séquence
        seq = seq_avant.lower()+motif.upper()+seq_apres.lower()
        
        posMotif.append(seq)
        motifs.append(motif)

 
    return posMotif, motifs

posMotif, motifs = createPosSequences(freq, nuc, k, 10, 30)

pos = [x.upper() for x in posMotif] #transform toutes les séquences en majuscules 


In [127]:
print(posMotif)
print(motifs)

['gtgaaaataatgtggtatgttAGTATAGcg', 'cggggcagaaatcttAGTATAGttcggtag', 'cAGTATAGagaattataaaccgcggctaat', 'ttaacagttcctaaggAGTATAGtacgggc', 'ccgcaagctgaAGTATAGagaaaacatacg', 'cggtagacAGTATAGtgcatggatgtaccg', 'AGTATAGtacggataacacataacgtgctg', 'gggtcgggatcggAGTATAGacccctaggg', 'cccagtAGTATAGgatccactgtataacgc', 'ctggcctgAGTATAGccgtcgtagtcatgt']
['AGTATAG', 'AGTATAG', 'AGTATAG', 'AGTATAG', 'AGTATAG', 'AGTATAG', 'AGTATAG', 'AGTATAG', 'AGTATAG', 'AGTATAG']


<b> 2. Générer la matrice de poids position</b> à partir de séquences générer artificiellement `motifs` 
puis comparer la matrice générée avec la matrice originale `freq`, que remarquez vous?

In [99]:
#Computing wi(a)
def matricePoidsPosition(motifs, nuc, k):
    """
    Genere la matrice de poids position à partir de motifs
    entrée motifs : list  contenant les motifs d'interets
    entrée nuc  : alphabet 
    entrée k    : taille du motif
    sortie pwm  : matrice de poids position qui represent les motifs
    """
    q = len(nuc)
    PWM = np.ones((q, k))

    #Parcours sur chaque séquence de la liste
    for m in motifs: 
        #Parcours sur la longueur de la séquence
        for i in range(k):
            #Incrémentation des occurences de l'alpahbet en fonction de la position
            PWM[nuc.index(m[i]),i] += 1 
    

    #Normalisation    
    PWM /= (len(motifs)+len(nuc))
    
    return PWM


PWM = matricePoidsPosition(motifs, nuc, k); 

print("PWM\n")
print (PWM)

print("\nfreq normalisé\n")
print(freq/1020)

print("\nOn peut quand même constater la présence du motif mais moins marqué\n")


PWM

[[0.78571429 0.07142857 0.07142857 0.78571429 0.07142857 0.78571429
  0.07142857]
 [0.07142857 0.07142857 0.07142857 0.07142857 0.07142857 0.07142857
  0.07142857]
 [0.07142857 0.78571429 0.07142857 0.07142857 0.07142857 0.07142857
  0.78571429]
 [0.07142857 0.07142857 0.78571429 0.07142857 0.78571429 0.07142857
  0.07142857]]

freq normalisé

[[0.98039216 0.01176471 0.00294118 0.98039216 0.00490196 0.98039216
  0.00098039]
 [0.00980392 0.00784314 0.00294118 0.00294118 0.         0.00980392
  0.00098039]
 [0.00392157 0.98039216 0.00196078 0.00686275 0.00490196 0.0127451
  0.98039216]
 [0.00588235 0.00980392 0.98039216 0.00490196 0.98039216 0.00588235
  0.00098039]]

On peut quand même constater la présence du motif mais moins marqué



<b>3. Chercher les motifs en utilisant l'algorithme `motifSearch` </b>:
<br>
Faire une fonction pour chercher le motif le plus plus frequents dans l'ensemble de sequences positives `pos`. 

In [105]:
def motifSearch(k:int, sequences:list):
    """
    Cherche les motifs de taille k dans un ensemble de séquences
    entrée k : taille du motif
    entrée séquences : liste de sequences
    sortie motifs: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    >>>searchMotifs(3, ['TAAGTAA', 'TATAA', 'CTATC'])
    {'TAA': 3, 'AAG': 1, 'AGT': 1, 'GTA': 1, 'TAT': 2, 'ATA': 1, 'CTA': 1, 'ATC': 1}
    """
    motifsFound  = {};
    liste = []
    for s in sequences:
        for j in range(len(s)-k+1):
            liste.append(s[j:k+j])

    for i in liste :
        if i in motifsFound.keys():
            motifsFound[i]+=1
        else:
            motifsFound[i]=1
        
    return motifsFound


def getTopMotifs(motifs:dict(), top=1):
    """
    renvoyer les top motifs le plus frequent
    entrée motifsFound: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    entrée top : les top plus frequent 
    sortie motifsfreq: dictionnaire contenant les top motifs les plus fréquents, clé=motif, valeur = fréquence d'observation
    >>>getTopMotifs({'TAA': 3, 'AAG': 1, 'AGT': 1, 'GTA': 1, 'TAT': 2, 'ATA': 1, 'CTA': 1, 'ATC': 1}, 2)
    {'TAA': 3, 'TAT': 2}
    """
    motifsfreq  = {}
    for i in range(top):
        top_in = max(motifs.values())
        k = [z for z in motifs.keys() if motifs[z]==top_in]
        if len(k)>top:
            for k_in in range(top):          
                if k[k_in] not in motifsfreq.keys():
                    motifsfreq[k[k_in]]=motifs[k[k_in]]
                    motifs[k[k_in]] = 0
                    break
        
        if len(k) == 1 : 
            l = k[0] 
            if l not in motifsfreq.keys():
                    motifsfreq[l]=motifs[l]
                    motifs[l] = 0

    return motifsfreq

motifsFound = motifSearch(k, pos)
topMotifs = getTopMotifs(motifsFound, 1)
print (topMotifs)


{'AGTATAG': 10}


4. Avez vous trouvez le motif N fois dans l'ensemble de séquences positives `pos`? Justifiez votre réponse.

<font color="#0000FF"> Réponse: Oui, car il est discernable et ne présente pas beaucoup de variabilité </font>

5. Utiliser l'algorithme Expectation-Maximisation pour chercher un motif de taille k dans l'ensemble de séquences `pos`.

In [138]:
def initialiseP(seqs, w, alph):
    """
    Initialise pc,k
    input seqs : list of sequences
    input w : motif length
    input alph : alphabet (nucleotides or amino acids)
    output P: position probability matrix
    """
    q=len(alph)
    P = np.zeros((q, w+1))
    # initialisation du model nul
    P[:, 0] = [1/q] * q
    # on tire une séquence de façon aléatoire
    rand_index_seq = np.random.randint(0, len(seqs))
    sequence = seqs[rand_index_seq]
    # on tire un motif de façon aléatoire
    rand_index_motif = np.random.randint(0, len(sequence) - w)
    motif = sequence[rand_index_motif:rand_index_motif + w]
    # on remplit les colones restantes
    for i in range(len(motif)):
        # la case du caractère du motif est mise à 0.5
        c_k = alph.index(motif[i])
        P[c_k][i + 1] = 0.5
        
        # les autres sont remplis équitablement de sorte à ce que la colonne somme à 1
        for j in range(q):
            if c_k != j:
                P[j][i + 1] = (1 - 0.5) / (q - 1)
    return P

def initialiseZ(seqs, w):
    """
    Initialise Z
    input seqs : list of sequences
    input w : motif length
    output Z :  matrix of motif start positions
    """
    n = len(seqs)
    L = len(seqs[0])
    Z = np.ones((n, (L - w + 1)))
    return Z

def compute_Z_for_seq(seq, P, w, alph):
    res = []
    # parcours des positions de la séquence
    for i in range(len(seq) - w + 1):
        # motif courrant
        motif = seq[i: i + w]
        # calcul de la probabilité du motif
        proba = 1
        for i in range(w):
            c_k = alph.index(motif[i])
            proba = proba * P[c_k][i + 1]
        # calcul des probas hors-motif (on suppose que le model nul est systématiquement equiprobable)
        proba_nulle = P[0][0] 
        proba = proba * pow(proba_nulle, len(seq) - w)
        
        res.append(proba)
    return res

def E_step(seqs, P, w, alph):
    """
    Implement Expectation step
    input seqs : list of sequences
    input P : position probability matrix
    input Z :  matrix of motif start positions
    input w : motif length
    input alph : alphabet (nucleotides or amino acids)
    output Z :  matrix of motif start positions
    """
    
    Z = initialiseZ(seqs, w)
    # on appelle la 
    for i in range(len(Z)):
        Z[i] = compute_Z_for_seq(seqs[i], P, w, alph)
    return Z

def normaliseZ(z):
    """
    Normalise Z matrix
    input Z : unnormalised matrix
    output Zn : normalised matrix
    """
    return np.array([row/sum(row) for row in z])

def M_step(seqs, Z, w, alph):
    """
    Implement Expectation step
    input seqs : list of sequences
    input Z :  matrix of motif start positions
    input w : motif length
    input alph : alphabet (nucleotides or amino acids)
    output P : position probability matrix
    """
    n = len(seqs)
    L = len(seqs[0])
    q = len(alph)
    
    # initialisation de P (avec pseudo-count)
    P = np.ones((q, w+1))
    
    # on va parcourir tous les motifs de toutes les séquences et ajouter
    # au fur et à mesure les probabilités du motif dans les cases
    # appropriées de P
    for i in range(n):
        for j in range(L - w + 1):
            # on récupère le motif
            motif = seqs[i][j:j + w]
            # parcours du motif
            for k in range(len(motif)):
                # caractère courant
                c_k = alph.index(motif[k])
                # position du caractère dans le motif
                pos = k + 1
                # ajout de la probabilité Z_i,j
                P[c_k][pos] += Z[i][j]
                
    # normalisation des colonnes (avec pseudo-count) : division par somme des Z + longueur alphabet
    # hors la somme des Z vaut le nombre de séquences, puisque chaque séquence a ses Z qui somment à 1
    P = P / (q + n)
    
    # ajout du model nul
    P[:, 0] = [1/q] * q
    return P


def ExpectationMaximisation(seqs, w, alph, eps):
    """
    Implement Expectation Maximisation algorithm
    input seqs : list of sequences
    input w : motif length
    input alph : alphabet (nucleotides or amino acids)
    input eps : threahold 
    output P : position probability matrix
    output Z :  matrix of motif start positions
    output lLikelihood : log likelihood of P 
    output pos_motif : positions of motifs in seqs
    """
    P = initialiseP(seqs, w, alph)
    Z = initialiseZ(seqs, w)
    lLikelihood = []
    pos_motif = []
    
    #Infini pour éviter les erreurs
    stop = np.inf
    
    #Boucler jusqu'à atteindre epsilon
    while stop>=eps:
        #Expectation
        Z = E_step(seqs,P,Z,w,alph)
        
        #Maximization
        P = M_step(seqs,Z,w,alph)
        
        #Calcul du log-vraisemblance et mise a jour du stop
        lLikelihood.append(likelihood(seqs,Z,P,w,alph))
        if len(lLikelihood)>=2:
            stop = np.abs(lLikelihood[len(lLikelihood)-2] - lLikelihood[len(lLikelihood)-1])
    
    #Récupération des meilleurs positions de motif
    for s in range(len(seqs)):
        pos_motif.append(np.argmax(Z[s]))

    return (P, lLikelihood, Z, pos_motif)

In [131]:
P, top_lLikelihood, Z,pos_motif = ExpectationMaximisation(pos,k,nuc,1e-4)

In [142]:
list_mot_par_EM = []
for i in range(len(pos_motif)):
    list_mot_par_EM.append(posMotif[i][pos_motif[i]:pos_motif[i]+k])

print(list_mot_par_EM)

['gaaaata', 'ggcagaa', 'GTATAGa', 'gttccta', 'gcaagct', 'gtagacA', 'GTATAGt', 'gtcggga', 'gtAGTAT', 'gcctgAG']


6. Comparez les motifs trouvés par EM et l'algorithme `motifSearch` avec les motifs `posMotif`, 
où les vrais motifs sont montrés en lettre majuscule. Quelle est la performance de ces deux programmes? 

In [143]:
motifsFound_LAST = motifSearch(k, posMotif)
topMotifs_LAST = getTopMotifs(motifsFound_LAST, 1)
print(topMotifs_LAST)

{'AGTATAG': 10}


In [144]:
print("L'algo EM ne fonctionne pas bien,  du coup j'ai une meilleure performance côté algorithme MotifsSearch")

L'algo EM ne fonctionne pas bien,  du coup j'ai une meilleure performance côté algorithme MotifsSearch
